In [ ]:
!pip install ultralytics

In [ ]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [1]:
from ultralytics import YOLO
import os
from IPython.display import display, Image
from IPython import display
display.clear_output()
!yolo checks



Ultralytics YOLOv8.2.95 🚀 Python-3.12.4 torch-2.4.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1660 Ti, 6144MiB)
Setup complete ✅ (12 CPUs, 31.9 GB RAM, 357.5/475.6 GB disk)

OS                  Windows-11-10.0.22631-SP0
Environment         Windows
Python              3.12.4
Install             pip
RAM                 31.91 GB
CPU                 Intel Core(TM) i7-10750H 2.60GHz
CUDA                11.8

numpy               ✅ 1.26.4<2.0.0,>=1.23.0
matplotlib          ✅ 3.8.4>=3.3.0
opencv-python       ✅ 4.10.0.84>=4.6.0
pillow              ✅ 10.2.0>=7.1.2
pyyaml              ✅ 6.0.1>=5.3.1
requests            ✅ 2.32.2>=2.23.0
scipy               ✅ 1.13.1>=1.4.1
torch               ✅ 2.4.1+cu118>=1.8.0
torchvision         ✅ 0.19.1+cu118>=0.9.0
tqdm                ✅ 4.66.4>=4.64.0
psutil              ✅ 5.9.0
py-cpuinfo          ✅ 9.0.0
pandas              ✅ 2.2.2>=1.1.4
seaborn             ✅ 0.13.2>=0.11.0
ultralytics-thop    ✅ 2.0.6>=2.0.0
torch               ✅ 2.4.1+cu118!=2.4.0,>=1.8.0; sys_p

### Process KangaiSet+Manga109

In [2]:
import os
import shutil
import random
import pandas as pd

emotion_map = {
    'anger': 0,
    'disgust': 1,
    'fear': 2,
    'happiness': 3,
    'neutral': 4,
    'sadness': 5,
    'surprise': 6
}
width = 1654
height = 1170

def create_yolo_dataset_from_csv(csv_file_path, output_dir='../KangaiSet_YOLO'):
    # read csv 
    df = pd.read_csv(csv_file_path)

    # create dirs
    for dataset_type in ['train', 'val', 'test']:
        os.makedirs(f'{output_dir}/{dataset_type}/images', exist_ok=True)
        os.makedirs(f'{output_dir}/{dataset_type}/labels', exist_ok=True)

    for index, row in df.iterrows():
        manga_name = row['manga_name']

        page_path = row['page_path']
        last_part = page_path.split('/')[-1]
        pg = last_part.split('.')[0]
        
        emotion_str = row['emotion_str']

        # if page_path == 'images/AisazuNihaIrarenai/004.jpg':
        #     print(emotion_str)

        # extract bounding box for face
        face_xmin = row['face_xmin']
        face_ymin = row['face_ymin']
        face_xmax = row['face_xmax']
        face_ymax = row['face_ymax']

        dataset_type = random.choices(['train', 'val', 'test'], weights=[0.85, 0.1, 0.05])[0]

        # copy image to corresponding manga
        image_source_path = f'../Manga109/{page_path}'
        image_dest_path = f'{output_dir}/{dataset_type}/images/{manga_name}_{pg}.jpg'
        shutil.copyfile(image_source_path, image_dest_path)

        label_file_path = f'{output_dir}/{dataset_type}/labels/{manga_name}_{pg}.txt'
        # write face bounding box
        with open(label_file_path, 'a+') as label_file:
            label_file.seek(0, 2)
            face_bbox = (
                (face_xmin + face_xmax) / (2 * width),
                (face_ymin + face_ymax) / (2 * height),
                (face_xmax - face_xmin) / width,
                (face_ymax - face_ymin) / height
            )
            label_file.write(f"{emotion_map[emotion_str]} {' '.join(map(str, face_bbox))}\n")

        counter = 0
        counter += 1
        if counter % 50 == 0:
            print(f'Processed {counter} images')

def main():
    csv_file_path = '../Manga109/kangaiset_annotation_long.csv'
    create_yolo_dataset_from_csv(csv_file_path)

if __name__ == '__main__':
    main()

### Train

In [3]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
!yolo task=detect mode=train model=yolov8n.pt data=kangaiset.yaml epochs=50 batch=16 device=cuda dropout=0.5 val=True save=True

^C


In [ ]:
# RE-TRAIN
!yolo task=detect mode=train model='runs/detect/train2/weights/best.pt' data=kangaiset.yaml epochs=50 batch=16 device=cuda dropout=0.5 val=True save=True

### Predict

In [ ]:
# test predict w/ best.pt
!yolo predict model='runs/detect/train2/weights/best.pt' source='test_page.jpeg' device=cuda